In [1]:
!wget https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv

--2024-11-02 14:14:44--  https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/jamb_exam_results.csv [following]
--2024-11-02 14:14:45--  https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/jamb_exam_results.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 391501 (382K) [text/plain]
Saving to: ‘jamb_exam_results.csv’

jamb_exam_results.c 100%[===================>] 382.33K  --.-KB/s    in 0.007s  

2024-11-02 14:14:45 (51.8 MB/s) - ‘jamb_exam_result

In [50]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_csv('/workspaces/ml-zoomcamp/06-trees/jamb_exam_results.csv')

In [4]:
df.head()

,JAMB_Score,Study_Hours_Per_Week,Attendance_Rate,Teacher_Quality,Distance_To_School,School_Type,School_Location,Extra_Tutorials,Access_To_Learning_Materials,Parent_Involvement,IT_Knowledge,Student_ID,Age,Gender,Socioeconomic_Status,Parent_Education_Level,Assignments_Completed
0,192,22,78,4,12.4,Public,Urban,Yes,Yes,High,Medium,1,17,Male,Low,Tertiary,2
1,207,14,88,4,2.7,Public,Rural,No,Yes,High,High,2,15,Male,High,NaN,1
2,182,29,87,2,9.6,Public,Rural,Yes,Yes,High,Medium,3,20,Female,High,Tertiary,2
3,210,29,99,2,2.6,Public,Urban,No,Yes,Medium,High,4,22,Female,Medium,Tertiary,1
4,199,12,98,3,8.8,Public,Urban,No,Yes,Medium,Medium,5,22,Female,Medium,Tertiary,1


In [5]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

**Preparation:**

- Remove the `student_id` column.
- Fill missing values with zeros.
- Do train/validation/test split with 60%/20%/20% distribution.
- Use the `train_test_split` function and set the `random_state` parameter to `1`.
- Use `DictVectorizer(sparse=True)` to turn the dataframes into matrices.


In [7]:
df.drop(columns=['student_id'], axis = 1, inplace=True)

In [13]:
df.isnull().sum()

jamb_score                        0
study_hours_per_week              0
attendance_rate                   0
teacher_quality                   0
distance_to_school                0
school_type                       0
school_location                   0
extra_tutorials                   0
access_to_learning_materials      0
parent_involvement                0
it_knowledge                      0
age                               0
gender                            0
socioeconomic_status              0
parent_education_level          891
assignments_completed             0
dtype: int64

In [16]:
df.fillna(0, inplace=True)

In [17]:
df.isnull().sum()

jamb_score                      0
study_hours_per_week            0
attendance_rate                 0
teacher_quality                 0
distance_to_school              0
school_type                     0
school_location                 0
extra_tutorials                 0
access_to_learning_materials    0
parent_involvement              0
it_knowledge                    0
age                             0
gender                          0
socioeconomic_status            0
parent_education_level          0
assignments_completed           0
dtype: int64

In [44]:
SEED = 1

In [45]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=SEED)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=SEED)

assert len(df) == (len(df_train) + len(df_val) + len(df_test))

In [46]:
len(df_train), len(df_val), len(df_test)

(3000, 1000, 1000)

In [47]:
X_train = df_train.drop(columns=['jamb_score']).to_dict(orient='records')
X_val = df_val.drop(columns=['jamb_score']).to_dict(orient='records')
X_test = df_test.drop(columns=['jamb_score']).to_dict(orient='records')

y_train = df_train['jamb_score']
y_val = df_val['jamb_score']
y_test = df_test['jamb_score']

dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(X_train)
X_val = dv.transform(X_val)
X_test = dv.transform(X_test)


**Question 1**

Let's train a decision tree regressor to predict the `jamb_score` variable.

- Train a model with `max_depth=1`.

Which feature is used for splitting the data?



In [48]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor(max_depth=1, random_state=SEED)

dt.fit(X_train, y_train)

feature_index = dt.tree_.feature[0]
feature_name = dv.feature_names_[feature_index]
feature_name


'study_hours_per_week'

In [49]:
for index in dt.tree_.feature:
    if index != -2:  # -2 is a leaf node
        print(dv.feature_names_[index])

study_hours_per_week


**Question 2**

Train a random forest model with these parameters:

- `n_estimators=10`
- `random_state=1`
- `n_jobs=-1` (optional - to make training faster)

What's the RMSE of this model on validation?



In [51]:
rf = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)

rf.fit(X_train, y_train)

y_pred_val = rf.predict(X_val)

rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
rmse_val


np.float64(42.13724207871227)

**Question 3**

Now let's experiment with the `n_estimators` parameter

- Try different values of this parameter from 10 to 200 with step 10.
- Set `random_state` to `1`.
- Evaluate the model on the validation dataset.

After which value of `n_estimators` does RMSE stop improving? Consider 3 decimal places for calculating the answer.


In [57]:
n_estimators_range = range(10, 220, 10)
rmse_values = []

for n in n_estimators_range:
    rf = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    rf.fit(X_train, y_train)
    
    y_pred_val = rf.predict(X_val)
    
    rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
    rmse_values.append((n, rmse_val))

for n, rmse in rmse_values:
    print(f"n_estimators: {n}, RMSE: {rmse:.3f}")

threshold = 0.001  
prev_rmse = rmse_values[0][1]
print(f"Initial RMSE: {prev_rmse:.3f}")  # Debugguing print

for i, (n, rmse) in enumerate(rmse_values[1:], start=1):
    print(f"Comparing prev_rmse: {prev_rmse:.3f} with current rmse: {rmse:.3f}")  # Debugging print
    if abs(prev_rmse - rmse) < threshold:
        print(f"RMSE stops improving after n_estimators = {n}")
        break
    prev_rmse = rmse


n_estimators: 10, RMSE: 42.137
n_estimators: 20, RMSE: 41.461
n_estimators: 30, RMSE: 41.106
n_estimators: 40, RMSE: 40.917
n_estimators: 50, RMSE: 40.852
n_estimators: 60, RMSE: 40.784
n_estimators: 70, RMSE: 40.677
n_estimators: 80, RMSE: 40.539
n_estimators: 90, RMSE: 40.504
n_estimators: 100, RMSE: 40.517
n_estimators: 110, RMSE: 40.593
n_estimators: 120, RMSE: 40.625
n_estimators: 130, RMSE: 40.651
n_estimators: 140, RMSE: 40.595
n_estimators: 150, RMSE: 40.597
n_estimators: 160, RMSE: 40.604
n_estimators: 170, RMSE: 40.628
n_estimators: 180, RMSE: 40.641
n_estimators: 190, RMSE: 40.631
n_estimators: 200, RMSE: 40.601
n_estimators: 210, RMSE: 40.520
Initial RMSE: 42.137
Comparing prev_rmse: 42.137 with current rmse: 41.461
Comparing prev_rmse: 41.461 with current rmse: 41.106
Comparing prev_rmse: 41.106 with current rmse: 40.917
Comparing prev_rmse: 40.917 with current rmse: 40.852
Comparing prev_rmse: 40.852 with current rmse: 40.784
Comparing prev_rmse: 40.784 with current rmse:

**Question 4**

Let's select the best `max_depth`:

- Try different values of `max_depth`: [10, 15, 20, 25]
- For each of these values,
- try different values of `n_estimators` from 10 till 200 (with step 10)
- calculate the mean RMSE
- Fix the random seed: `random_state=1`

What's the best `max_depth`, using the mean RMSE?



In [58]:
max_depth_values = [10, 15, 20, 25]
n_estimators_range = range(10, 210, 10)
random_state = 1

mean_rmse_results = {}

for max_depth in max_depth_values:
    rmse_list = []
    
    for n_estimators in n_estimators_range:
        rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state, n_jobs=-1)
        rf.fit(X_train, y_train)
        
        y_pred_val = rf.predict(X_val)
        rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
        rmse_list.append(rmse_val)
    
    mean_rmse = np.mean(rmse_list)
    mean_rmse_results[max_depth] = mean_rmse

for depth, mean_rmse in mean_rmse_results.items():
    print(f"max_depth: {depth}, Mean RMSE: {mean_rmse:.3f}")

best_max_depth = min(mean_rmse_results, key=mean_rmse_results.get)
best_mean_rmse = mean_rmse_results[best_max_depth]

print(f"Best max_depth: {best_max_depth} with Mean RMSE: {best_mean_rmse:.3f}")


max_depth: 10, Mean RMSE: 40.392
max_depth: 15, Mean RMSE: 40.735
max_depth: 20, Mean RMSE: 40.740
max_depth: 25, Mean RMSE: 40.788
Best max_depth: 10 with Mean RMSE: 40.392


**Question 5**

We can extract feature importance information from tree-based models.

At each step of the decision tree learning algorithm, it finds the best split. When doing it, we can calculate "gain" - the reduction in impurity before and after the split. This gain is quite useful in understanding what are the important features for tree-based models.

In Scikit-Learn, tree-based models contain this information in the feature_importances_ field.

For this homework question, we'll find the most important feature:
<ul>
<li>Train the model with these parameters:</li>
<ul>
<li><code>n_estimators=10</code>,</li>
<li><code>max_depth=20</code>,</li>
<li><code>random_state=1</code>,</li>
<li><code>n_jobs=-1 (optional)</code></li>
</ul>
</ul>
Get the feature importance information from this model

What's the most important feature (among these 4)?



In [59]:
n_estimators = 10
max_depth = 20
random_state = 1

rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state, n_jobs=-1)
rf.fit(X_train, y_train)

importances = rf.feature_importances_

feature_names = dv.get_feature_names_out()

feature_importance_list = list(zip(feature_names, importances))

sorted_features = sorted(feature_importance_list, key=lambda x: x[1], reverse=True)

for feature, importance in sorted_features:
    print(f"Feature: {feature}, Importance: {importance:.4f}")

most_important_feature = sorted_features[0]
print(f"\nMost important feature: {most_important_feature[0]} with importance: {most_important_feature[1]:.4f}")


Feature: study_hours_per_week, Importance: 0.2484
Feature: attendance_rate, Importance: 0.1497
Feature: distance_to_school, Importance: 0.1365
Feature: teacher_quality, Importance: 0.0827
Feature: age, Importance: 0.0693
Feature: assignments_completed, Importance: 0.0315
Feature: socioeconomic_status=High, Importance: 0.0257
Feature: parent_involvement=High, Importance: 0.0229
Feature: it_knowledge=High, Importance: 0.0177
Feature: parent_education_level=Secondary, Importance: 0.0170
Feature: parent_education_level=Primary, Importance: 0.0155
Feature: parent_education_level=Tertiary, Importance: 0.0145
Feature: extra_tutorials=No, Importance: 0.0135
Feature: parent_involvement=Low, Importance: 0.0134
Feature: it_knowledge=Low, Importance: 0.0124
Feature: access_to_learning_materials=No, Importance: 0.0123
Feature: parent_involvement=Medium, Importance: 0.0115
Feature: socioeconomic_status=Low, Importance: 0.0107
Feature: socioeconomic_status=Medium, Importance: 0.0106
Feature: gender=M

WE can go more specific using the code below

In [60]:
n_estimators = 10
max_depth = 20
random_state = 1

rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state, n_jobs=-1)
rf.fit(X_train, y_train)

importances = rf.feature_importances_

feature_names = dv.get_feature_names_out()

feature_importance_dict = dict(zip(feature_names, importances))

specific_features = ['study_hours_per_week', 'attendance_rate', 'distance_to_school', 'teacher_quality']

specific_importances = {feature: feature_importance_dict.get(feature, 0) for feature in specific_features}

most_important_feature = max(specific_importances, key=specific_importances.get)
most_important_importance = specific_importances[most_important_feature]

print("Feature Importances:")
for feature, importance in specific_importances.items():
    print(f"{feature}: Importance = {importance:.4f}")

print(f"\nMost Important Feature: {most_important_feature} with Importance = {most_important_importance:.4f}")


Feature Importances:
study_hours_per_week: Importance = 0.2484
attendance_rate: Importance = 0.1497
distance_to_school: Importance = 0.1365
teacher_quality: Importance = 0.0827

Most Important Feature: study_hours_per_week with Importance = 0.2484


**Question 6**

Now let's train an XGBoost model! For this question, we'll tune the eta parameter:

- Install XGBoost
- Create DMatrix for train and validation
- Create a watchlist
- Train a model with these parameters for 100 rounds:
```
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
```
Now change `eta` from `0.3` to `0.1`.

Which eta leads to the best RMSE score on the validation dataset?



In [61]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 71.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 MB 46.8 MB/s eta 0:00:0000:0100:01


In [62]:
import xgboost as xgb

# Define XGBoost parameters for eta = 0.3
xgb_params_0_3 = {
    'eta': 0.3,
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model_0_3 = xgb.XGBRegressor(**xgb_params_0_3, n_estimators=100)

model_0_3.fit(X_train, y_train)

y_pred_val_0_3 = model_0_3.predict(X_val)

rmse_0_3 = np.sqrt(mean_squared_error(y_val, y_pred_val_0_3))
print(f"RMSE for eta = 0.3: {rmse_0_3:.4f}")

# Define XGBoost parameters for eta = 0.1
xgb_params_0_1 = {
    'eta': 0.1,
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model_0_1 = xgb.XGBRegressor(**xgb_params_0_1, n_estimators=100)

model_0_1.fit(X_train, y_train)

y_pred_val_0_1 = model_0_1.predict(X_val)

rmse_0_1 = np.sqrt(mean_squared_error(y_val, y_pred_val_0_1))
print(f"RMSE for eta = 0.1: {rmse_0_1:.4f}")

# Compare RMSE results
if rmse_0_3 < rmse_0_1:
    print("Best eta is 0.3")
else:
    print("Best eta is 0.1")

RMSE for eta = 0.3: 43.4188
RMSE for eta = 0.1: 41.0503
Best eta is 0.1
